In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [309]:
train=pd.read_csv('data/train.csv')
test=pd.read_csv('data/test.csv')

In [310]:
print(train.head())
print(train.shape)

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  
(8

In [311]:
to_drop=['PassengerId','Name','Ticket']
test_y_id=test['PassengerId']

In [312]:
train=train.drop(columns=to_drop)
test=test.drop(columns=to_drop)

In [313]:
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,NaN,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,NaN,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,NaN,S


In [314]:
test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,male,34.5,0,0,7.8292,NaN,Q
1,3,female,47.0,1,0,7.0000,NaN,S
2,2,male,62.0,0,0,9.6875,NaN,Q
3,3,male,27.0,0,0,8.6625,NaN,S
4,3,female,22.0,1,1,12.2875,NaN,S


In [315]:
train=train.fillna(train.mean())
test=test.fillna(test.mean())

In [316]:
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,NaN,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,NaN,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,NaN,S


In [317]:
train[['Cabin']]=train[['Cabin']].fillna(train.mode()['Cabin'][0])
test[['Cabin']]=test[['Cabin']].fillna(train.mode()['Cabin'][0]) 


train[['Embarked']]=train[['Embarked']].fillna(train.mode()['Embarked'][0])
test[['Embarked']]=test[['Embarked']].fillna(train.mode()['Embarked'][0]) 

In [318]:
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,B96 B98,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,B96 B98,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,B96 B98,S


In [319]:
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null object
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Cabin       891 non-null object
Embarked    891 non-null object
dtypes: float64(2), int64(4), object(3)
memory usage: 52.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
Pclass      418 non-null int64
Sex         418 non-null object
Age         418 non-null float64
SibSp       418 non-null int64
Parch       418 non-null int64
Fare        418 non-null float64
Cabin       418 non-null object
Embarked    418 non-null object
dtypes: float64(2), int64(3), object(3)
memory usage: 21.3+ KB


In [320]:
from sklearn import preprocessing
le=preprocessing.LabelEncoder()

In [321]:
train_x=train.iloc[:,1:]
train_y=train['Survived']
test_x=test

In [322]:
col=['Sex','Cabin','Embarked']
for i in col:
    train_x[i]=le.fit_transform(train_x[i])
    test_x[i]=le.fit_transform(test_x[i])

In [323]:
print(train_x.shape,test_x.shape)

(891, 8) (418, 8)


In [324]:
train_x.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,1,22.0,1,0,7.2500,47,2
1,1,0,38.0,1,0,71.2833,81,0
2,3,0,26.0,0,0,7.9250,47,2
3,1,0,35.0,1,0,53.1000,55,2
4,3,1,35.0,0,0,8.0500,47,2


## Decision Tree Classifier

at each node of the decision tree select the feature which has highest information gain
Entropy defines the randomness

In [325]:

def entropy(Y):
    x=np.unique(np.array(Y),return_counts=True)
    e=0.0
    for i in x[1]:
        p=i*1.0/Y.shape[0]
        e+=(-1.0*p)*np.log2(p)
    return e

In [326]:
def divide(X,Y,feature,fval):
    left_x=pd.DataFrame([],columns=X.columns)
    right_x=pd.DataFrame([],columns=X.columns)
    
    left_y=pd.DataFrame([])
    right_y=pd.DataFrame([])
    
    for i in range(X.shape[0]):
        if X[feature].iloc[i]<=fval:
            left_x=left_x.append(X.iloc[i])
            left_y=left_y.append([Y.iloc[i]])
        else:
            #print(X.iloc[i])
            right_x=right_x.append(X.iloc[i])
            right_y=right_y.append([Y.iloc[i]])
    left_x.head()
    return left_x,right_x,left_y,right_y
def information_gain(X,Y,feature,fval):
    left_x,right_x,left_y,right_y=divide(X,Y,feature,fval)
    
    if left_x.shape[0]==Y.shape[0] or right_x.shape[0]==Y.shape[0]:
        return -1000000
    
    e=entropy(Y)
    
    e-=(1.0*(left_y.shape[0])/Y.shape[0])*entropy(left_y)
    e-=(1.0*(right_y.shape[0])/Y.shape[0])*entropy(right_y)
    return e
    
    
    
    

In [329]:
class DecisionTree:    
    def __init__(self,depth=0,max_depth=5):
        self.left=None
        self.right=None
        self.target=-1
        self.feature=None
        self.fval=None
        self.depth=depth
        self.max_depth=max_depth
    def Training(self,X,Y):
        
        #leaf node
        if self.depth==self.max_depth:
            self.target=Y.mode().iloc[0]
            return;
        
        
        l=[]
        for i in X.columns:
            l.append(information_gain(X,Y,i,X[i].mean()))
            #print(i)
        #print(l)
        
        
        
        self.feature=X.columns[np.argmax(l)]
        self.fval=X[self.feature].mean()
        
        #print(self.fval)
        
        
        left_x,right_x,left_y,right_y=divide(X,Y,self.feature,X[self.feature].mean())
        ##leaf node 
        if left_x.shape[0]==Y.shape[0] or right_x.shape[0]==Y.shape[0]:
            self.target=Y.mode().iloc[0]
            return;
        
        
        self.left=DecisionTree(self.depth+1,self.max_depth)
        self.left.Training(left_x,left_y)
        
        
        self.right=DecisionTree(self.depth+1,self.max_depth)
        self.right.Training(right_x,right_y)
        
        
        #print(np.argmax(l))
    def Predict(self,test_x):
        if self.left is None or self.right is None:
            return self.target
        if test_x[self.feature]<=self.fval:
            return self.left.Predict(test_x)
        else:
            return self.right.Predict(test_x)

obj=DecisionTree()
obj.Training(train_x,train_y)



In [330]:
predict_y=[]
for i in range(test_x.shape[0]):
    #print(test_x.loc[i])
    predict_y.append([test_y_id.iloc[i],obj.Predict(test_x.loc[i])[0]])
print(predict_y)

[[892, 0], [893, 0], [894, 0], [895, 0], [896, 1], [897, 0], [898, 1], [899, 0], [900, 1], [901, 0], [902, 0], [903, 0], [904, 1], [905, 0], [906, 1], [907, 1], [908, 0], [909, 0], [910, 0], [911, 1], [912, 0], [913, 0], [914, 1], [915, 0], [916, 1], [917, 0], [918, 1], [919, 0], [920, 0], [921, 0], [922, 0], [923, 0], [924, 0], [925, 0], [926, 0], [927, 0], [928, 0], [929, 0], [930, 0], [931, 0], [932, 1], [933, 0], [934, 0], [935, 1], [936, 1], [937, 0], [938, 0], [939, 0], [940, 1], [941, 1], [942, 0], [943, 0], [944, 1], [945, 1], [946, 0], [947, 0], [948, 0], [949, 1], [950, 0], [951, 1], [952, 0], [953, 0], [954, 0], [955, 1], [956, 0], [957, 1], [958, 1], [959, 0], [960, 0], [961, 1], [962, 1], [963, 0], [964, 0], [965, 1], [966, 1], [967, 0], [968, 0], [969, 1], [970, 0], [971, 1], [972, 1], [973, 0], [974, 0], [975, 0], [976, 0], [977, 0], [978, 1], [979, 0], [980, 1], [981, 1], [982, 1], [983, 0], [984, 1], [985, 0], [986, 0], [987, 0], [988, 1], [989, 0], [990, 0], [991, 0],

In [331]:
res=pd.DataFrame(np.array(predict_y),columns=['PassengerId','Survived'])

In [332]:
res.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [333]:
res.to_csv('res.csv',index=False)

## Inbuilt decision tree classifier

In [353]:
from sklearn.tree import  DecisionTreeClassifier
#DecisionTreeClassifier?
dtc=DecisionTreeClassifier()

In [354]:
dtc.fit(train_x,train_y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [355]:
inbuilt_predict_y=dtc.predict(test_x)

In [356]:

predict_y=[]
for i in range(inbuilt_predict_y.shape[0]):
    #print(test_x.loc[i])
    predict_y.append([test_y_id.iloc[i],inbuilt_predict_y[i]])
res=pd.DataFrame(np.array(predict_y),columns=['PassengerId','Survived'])
res.to_csv('res.csv',index=False)

In [357]:
dtc.score(train_x,train_y)


0.9865319865319865

## Random Forest Classifier

### We make two or more model by splitting the features or example of the training data.

### then we take the majority.

In [367]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(max_depth=5)

In [368]:
rfc

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [369]:
rfc.fit(train_x,train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [370]:
inbuilt_predict_y=rfc.predict(test_x)
predict_y=[]
for i in range(inbuilt_predict_y.shape[0]):
    #print(test_x.loc[i])
    predict_y.append([test_y_id.iloc[i],inbuilt_predict_y[i]])
res=pd.DataFrame(np.array(predict_y),columns=['PassengerId','Survived'])
res.to_csv('res.csv',index=False)

In [371]:
rfc.score(train_x,train_y)


0.8585858585858586